In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/identifying-disease-in-tea-leafs
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/Anthracnose
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/healthy
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/bird eye spot
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/brown blight
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/red leaf spot
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/gray light
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/algal leaf


In [7]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]

import tensorflow as tf
import keras
from keras import layers
from keras import ops

import numpy as np
import matplotlib.pyplot as plt

In [8]:
path = "/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset"

train,test = tf.keras.utils.image_dataset_from_directory(
    path,
    label_mode="categorical",
    validation_split=0.2,
    subset="both",
    seed=42,
    batch_size=32,
    image_size=(768, 768),
)

train=train.cache().prefetch(1).shuffle(100)
test=test.cache().prefetch(1)

Found 885 files belonging to 8 classes.
Using 708 files for training.
Using 177 files for validation.


In [9]:
# Convert to numpy arrays
x_train = []
y_train = []
for images, labels in train:
    x_train.append(images.numpy())
    y_train.append(labels.numpy())

x_train = tf.concat(x_train, axis=0).numpy()
y_train = tf.concat(y_train, axis=0).numpy()

x_test = []
y_test = []
for images, labels in test:
    x_test.append(images.numpy())
    y_test.append(labels.numpy())

x_test = tf.concat(x_test, axis=0).numpy()
y_test = tf.concat(y_test, axis=0).numpy()

print("Training set:", x_train.shape, y_train.shape)
print("Test set:", x_test.shape, y_test.shape)


Training set: (708, 768, 768, 3) (708, 8)
Test set: (177, 768, 768, 3) (177, 8)


In [10]:
# change y labels as class numbers instead of one-hot encoding
y_train_label = []
for y in y_train:
    label = np.argmax(y)
    y_train_label.append(label)

y_train_label = np.array(y_train_label).reshape(y_train.shape[0], 1)

y_test_label = []
for y in y_test:
    label = np.argmax(y)
    y_test_label.append(label)

y_test_label = np.array(y_test_label).reshape(y_test.shape[0], 1)

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train_label.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test_label.shape}")

x_train shape: (708, 768, 768, 3) - y_train shape: (708, 1)
x_test shape: (177, 768, 768, 3) - y_test shape: (177, 1)


In [11]:
learning_rate = 0.0003
weight_decay = 0.001
batch_size = 128
num_epochs = 100
image_size = 768
patch_size = 16
num_patches = (image_size // patch_size) ** 2

projection_dim = 128
num_heads = 8

transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 8
mlp_head_units = [
    1024,
    512,
]

In [13]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor = 0.2),
        layers.RandomContrast(factor=0.2),
    ],
    name = "data_augmentation"
)

data_augmentation.layers[0].adapt(x_train)

In [15]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activation.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x